In [1]:
import open3d as o3d
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
bad_pcds = []
for i in range(8):
    pcd = o3d.io.read_point_cloud(f"../seeds_scale/bad_pointclouds/pose_{i:02}_pcd.ply").farthest_point_down_sample(num_samples=1024)
    bad_pcds.append(pcd)

In [8]:
# Random pick some point as bad grasp
bad_conditions = []
bad_points = []
bad_pcd_label = []
bad_scores = np.zeros(240)
for i in range(8):
    bad_c_per_pcd = []
    points = np.asarray(bad_pcds[i].points)
    for j in range(30):
        bad_c_per_pcd.append(points[np.random.choice(len(points),2,replace=False)].flatten())
        bad_pcd_label.append(i+33)
        bad_points.append(points.copy())
    bad_conditions += bad_c_per_pcd
bad_conditions = np.vstack(bad_conditions)
bad_pcd_label = np.array(bad_pcd_label)
bad_points = np.asarray(bad_points)
print(bad_conditions.shape)
print(bad_pcd_label.shape)
print(bad_points.shape)

(240, 6)
(240,)
(240, 1024, 3)


In [9]:
np.savez("../score_function_data/scale/score_data_1540.npz", scores = bad_scores, point_clouds=bad_points, point_cloud_labels=bad_pcd_label, conditions=bad_conditions)